In [28]:
import pandas as pd
from datetime import datetime
import os, sys
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException, JavascriptException, UnexpectedAlertPresentException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select

# executable_path = r"C:\Users\817932702\mamikos\chromedriver\120\chromedriver.exe"
executable_path = r"D:\proyek\chromedriver\120\chromedriver.exe"
# executable_path = r'D:\proyek\scrapper\chromedriver\120\chromedriver.exe'
service = Service(executable_path)

options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False) 
options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(service=service, options=options)

# Fungsi untuk mensimulasikan scroll ke bawah halaman
def scroll_down():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


driver.get("https://mamikos.com/") 

time.sleep(5)

cari = driver.find_element(By. XPATH, '//*[@id="home"]/div[4]/div/div/div/div[1]')
cari.click()

time.sleep(2)

area = driver.find_element(By. XPATH, '//*[@id="home"]/div[13]/div/div[2]/div[2]/ul/li[2]/a')
area.click()

time.sleep(2)

kota = driver.find_element(By. XPATH, '//*[@id="home"]/div[13]/div/div[2]/div[4]/div/div[25]/details/summary')
kota.click()

time.sleep(2)

kota_2 = driver.find_element(By. XPATH, '//*[@id="home"]/div[13]/div/div[2]/div[4]/div/div[25]/details/div/a[5]')
kota_2.click()

time.sleep(5)

# oke_button = driver.find_element(By.XPATH, '//*[@id="desktopBannerWrapped"]/div/div[3]/div[1]/button[1]')
# oke_button.click()

list_df = []

# Inisialisasi indeks awal
start_index = 1

while True:
    try:
        # Coba temukan next_button
        next_button = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[2]/div/a')
        next_button.click()

        # Tunggu hingga halaman selanjutnya dimuat
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[1]/div[1]'))
        )

        # Scroll ke bawah untuk memuat lebih banyak data
        scroll_down()
        time.sleep(2)  # Tunggu sebentar setelah setiap scroll

        # Atur ulang indeks
        start_index += 20

    # except NoSuchElementException:
    #     # Jika next_button tidak ditemukan, keluar dari loop
    #     break
    except ElementNotInteractableException:
        # Tangani popup dengan menekan tombol OK
        alert_button_xpath = '//*[@id="filterKostTypeWrapper"]/div/div[1]/div[1]/div/div/div[3]/button'
        try:
            alert_button = driver.find_element(By.XPATH, alert_button_xpath)
            alert_button.click()
            print("Popup ditangani.")
        except NoSuchElementException:
            print("Tidak dapat menemukan tombol OK pada popup.")
            break
    except NoSuchElementException:
        break


Popup ditangani.


In [60]:
nama_kos = driver.find_elements(By.XPATH, '//*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[1]/div[*]/div/div/div/div[2]/div[2]/div/span[1]')
alamat_kos = driver.find_elements(By.XPATH, '//*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[1]/div[*]/div/div/div/div[2]/div[2]/div/span[2]')
fasilitas = driver.find_elements(By.XPATH, '//*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[1]/div[*]/div/div/div/div[2]/div[3]/div')
harga = driver.find_elements(By.XPATH, '//*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[1]/div[*]/div/div/div/div[2]/div[4]/div/div/div/span[1]')

# Buat list untuk menyimpan data
data = []

for nama, alamat, fasilitas, harga in zip(nama_kos, alamat_kos, fasilitas, harga):
    data.append({
        'Nama Kos': nama.text,
        'Alamat Kos': alamat.text,
        'Fasilitas': fasilitas.text,
        'Harga': harga.text
    })

    # Buat DataFrame dari list data
    df = pd.DataFrame(data)

In [61]:
df

,Nama Kos,Alamat Kos,Fasilitas,Harga
0,Kost Singgahsini Kemang 6 Tipe A Mampang Prapa...,Kecamatan Mampang Prapatan,K. Mandi Dalam ·WiFi ·AC ·Kloset Duduk ·Kasur ...,Rp2.675.000
1,Kost Senopati SCBD 2 Kebayoran Baru Jakarta Se...,Kebayoran Baru,K. Mandi Dalam ·WiFi ·AC ·Kloset Duduk ·Kasur ...,Rp1.800.000
2,Kost Apik Pondok Jaya VIII Tipe C Mampang Prap...,Mampang Prapatan,WiFi ·Kloset Duduk ·Kasur ·Akses 24 Jam\n4.5,Diskon 200rb sewa 3 bulan
3,Kost Singgahsini Kemang Timur Syariah Tipe B M...,Kecamatan Mampang Prapatan,K. Mandi Dalam ·WiFi ·AC ·Kloset Duduk ·Kasur ...,Rp1.325.000
4,Kost Apik Evy Kalibata Utara Tipe A Pancoran J...,Pancoran,K. Mandi Dalam ·WiFi ·Kasur ·Akses 24 Jam\n4.9,Diskon 200rb sewa 3 bulan
...,...,...,...,...
1795,Kost Sweet Home 1 Tipe G1 Pancoran Jakarta Sel...,Pancoran,WiFi ·Kasur,Rp5.200.000
1796,Kost Sweet Home 1 Tipe G2 Pancoran Jakarta Sel...,Pancoran,WiFi ·Kasur,Rp3.260.000
1797,Kost Biru Laut Tipe C Pancoran Jakarta Selatan,Pancoran,K. Mandi Dalam ·Kloset Duduk ·Kasur ·Akses 24 Jam,Rp3.000.000
1798,Kost Dnala Tipe Standar Mampang Prapatan Jakar...,Kecamatan Mampang Prapatan,K. Mandi Dalam ·WiFi ·AC ·Kloset Duduk ·Kasur ...,Rp2.550.000


In [46]:
max_columns = max(len(row) for row in list_df9)

# Mengisi kolom dengan data dan mengonversi ke DataFrame
df = pd.DataFrame(list_df9, columns=range(max_columns))

# Tampilkan DataFrame
print(df)

                0             1   
0          Campur  Sisa 1 kamar  \
1     Kos Andalan        Campur   
2          Campur  Sisa 2 kamar   
3          Campur  Sisa 1 kamar   
4           Putri  Sisa 2 kamar   
...           ...           ...   
1795  Kos Andalan         Putra   
1796  Kos Andalan         Putra   
1797  Kos Andalan        Campur   
1798  Kos Andalan        Campur   
1799  Kos Andalan        Campur   

                                                      2   
0     Kost Singgahsini Kemang 6 Tipe A Mampang Prapa...  \
1     Kost Senopati SCBD 2 Kebayoran Baru Jakarta Se...   
2     Kost Apik Pondok Jaya VIII Tipe C Mampang Prap...   
3     Kost Singgahsini Kemang Timur Syariah Tipe B M...   
4     Kost Apik Evy Kalibata Utara Tipe A Pancoran J...   
...                                                 ...   
1795  Kost Sweet Home 1 Tipe G1 Pancoran Jakarta Sel...   
1796  Kost Sweet Home 1 Tipe G2 Pancoran Jakarta Sel...   
1797     Kost Biru Laut Tipe C Pancoran Jakarta 

In [47]:
from IPython.display import display
display(df)

,0,1,2,3,4,5,6,7,8,9
0,Campur,Sisa 1 kamar,Kost Singgahsini Kemang 6 Tipe A Mampang Prapa...,Kecamatan Mampang Prapatan,K Mandi Dalam WiFi AC Kloset Duduk Kasur Akses...,48,Diskon 150rb,Rp2825000,Rp2675000,Bulan pertama
1,Kos Andalan,Campur,Kost Senopati SCBD 2 Kebayoran Baru Jakarta Se...,Kebayoran Baru,K Mandi Dalam WiFi AC Kloset Duduk Kasur Akses...,41,Rp1800000,bulan,None,None
2,Campur,Sisa 2 kamar,Kost Apik Pondok Jaya VIII Tipe C Mampang Prap...,Mampang Prapatan,WiFi Kloset Duduk Kasur Akses 24 Jam,45,Diskon 200rb sewa 3 bulan,Rp1325000,bulan,None
3,Campur,Sisa 1 kamar,Kost Singgahsini Kemang Timur Syariah Tipe B M...,Kecamatan Mampang Prapatan,K Mandi Dalam WiFi AC Kloset Duduk Kasur Akses...,46,Diskon 200rb sewa 3 bulan,Rp1925000,bulan,None
4,Putri,Sisa 2 kamar,Kost Apik Evy Kalibata Utara Tipe A Pancoran J...,Pancoran,K Mandi Dalam WiFi Kasur Akses 24 Jam,49,Diskon 200rb sewa 3 bulan,Rp1375000,bulan,None
...,...,...,...,...,...,...,...,...,...,...
1795,Kos Andalan,Putra,Kost Sweet Home 1 Tipe G1 Pancoran Jakarta Sel...,Pancoran,WiFi Kasur,Rp1200000,bulan,None,None,None
1796,Kos Andalan,Putra,Kost Sweet Home 1 Tipe G2 Pancoran Jakarta Sel...,Pancoran,WiFi Kasur,Rp1000000,bulan,None,None,None
1797,Kos Andalan,Campur,Kost Biru Laut Tipe C Pancoran Jakarta Selatan,Pancoran,K Mandi Dalam Kloset Duduk Kasur Akses 24 Jam,Rp1125000,bulan,None,None,None
1798,Kos Andalan,Campur,Kost Dnala Tipe Standar Mampang Prapatan Jakar...,Kecamatan Mampang Prapatan,K Mandi Dalam WiFi AC Kloset Duduk Kasur Akses...,Rp2500000,bulan,None,None,None
